In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/glassdoor_jobs.csv")
df = df.drop("Unnamed: 0", axis=1)
df.replace(-1, np.nan, inplace=True)
df.replace("-1", np.nan, inplace=True)

In [ ]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),NaN
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),NaN
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),NaN
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [ ]:
#Salary parsing
df = df.dropna(axis=0, subset=["Salary Estimate"])
df["hourly"] = df["Salary Estimate"].apply(lambda x: 1 if "per hour" in x.lower() else 0)

salary = df["Salary Estimate"].apply(lambda x: x.split("(")[0])
salary_range = salary.apply(lambda x: x.replace("K", "").replace("$", ""))
salary_range = salary_range.apply(lambda x: x.lower().replace("per hour", "").replace("employer provided salary:", ""))

# get min and max salary
df["min_salary"] = salary_range.apply(lambda x: float(x.split("-")[0]))
df["max_salary"] = salary_range.apply(lambda x: float(x.split("-")[1]))

# convert hourly salary to anual one
# note 8(hours) x 5(days per week) x 52(weeks per year) = 2080
df["min_salary"] = df.apply(lambda x: x["min_salary"]*2.08 if x["hourly"] else x["min_salary"], axis=1)
df["max_salary"] = df.apply(lambda x: x["max_salary"]*2.08 if x["hourly"] else x["max_salary"], axis=1)

df["avr_salary"] = (df["min_salary"] + df["max_salary"])/2

In [ ]:
#Company name without rating
df["company_txt"] = df.apply(lambda x: x["Company Name"] if x["Rating"]==1 else x["Company Name"][:-3], axis=1)
df["company_txt"] = df["company_txt"].apply(lambda x: x.replace("\n", ''))
df["company_txt"].nunique()

343

In [ ]:
# company age
df["age"] = 2023-df["Founded"]

In [ ]:
#State field
df["job_state"] = df["Location"].apply(lambda x: x.split(",")[1])
df["job_state"] = df["job_state"].apply(lambda x: x.strip() if x.strip().lower() != "los angeles" else "CA")
df["same_state"] = df["Location"] == df["Headquarters"]

df["job_state"].value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
NC     21
WA     21
NJ     17
FL     16
OH     14
TN     13
DC     11
CO     11
WI     10
UT     10
IN     10
AZ      9
MO      9
AL      8
GA      6
DE      6
KY      6
MI      6
CT      5
IA      5
LA      4
OR      4
NE      4
NM      3
KS      3
ID      2
MN      2
RI      1
SC      1
Name: job_state, dtype: int64

In [ ]:
#Required tools from job description
# python
# r-studio
# excel
# sql

df["Job Description"][0]

'Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relevant field, e.g., math, data analysis, database, computer science, Artificial Intelligence (AI); three years’ experience credit for Master’s degree; five years’ experience credit for a Ph.D\nApplicant should be proficient in the use of Power BI, Tableau, Python, MATLAB, Microsoft Word, PowerPoint, Excel, and working knowledge of MS Access, LMS, SAS, data visualization tools, and have a strong algorithmic aptitude\nExcellent verbal and written communication skills, and quantitative analytical skills are required\nApplicant must be able to work in a team environment\nU.S. citizenship and ability to obtain a DoD Secret Clearance required\nResponsibilities: The applicant will be responsible for formulating analytical solutions to complex data problems; creating data analytic models to improve data

In [ ]:
df["python_yn"] = df["Job Description"].apply(lambda x: "python" in x.lower())
df["python_yn"].value_counts()

True     392
False    350
Name: python_yn, dtype: int64

In [ ]:
df["R_yn"] = df["Job Description"].apply(lambda x: "r studio" in x.lower() or "r-studio" in x.lower())
df["R_yn"].value_counts()

False    740
True       2
Name: R_yn, dtype: int64

In [ ]:
df["sql_yn"] = df["Job Description"].apply(lambda x: "sql" in x.lower())
df["sql_yn"].value_counts()

True     380
False    362
Name: sql_yn, dtype: int64

In [ ]:
df["excel_yn"] = df["Job Description"].apply(lambda x: "excel" in x.lower())
df["excel_yn"].value_counts()

True     388
False    354
Name: excel_yn, dtype: int64

In [ ]:
# Title parsing, get title and seniority
def title_parse(title):
  if "data scientist" in title.lower() or "data science" in title.lower():
    return "data scientist"
  elif "data analyst" in title.lower() or "data analysis" in title.lower():
    return "data analyst"
  elif "data engineer" in title.lower():
    return "data engineer"
  elif "machine learning" in title.lower():
    return "machine learning"
  elif "manager" in title.lower():
    return "manager"
  elif "director" in title.lower():
    return "director"
  return "na"

df["job_title"]=df["Job Title"].apply(title_parse)
df["job_title"].value_counts()

data scientist      313
na                  167
data engineer       119
data analyst        100
machine learning     22
manager              16
director              5
Name: job_title, dtype: int64

In [ ]:
def seniority_parse(title):
  if "sr" in title.lower() or "senior" in title.lower() or "lead" in title.lower() or "principal" in title.lower():
    return "senior"
  elif "middle" in title.lower():
    return "junior"
  elif "jr" in title.lower() or "junior" in title.lower():
    return "junior"
  return "na"

df["seniority"]=df["Job Title"].apply(seniority_parse)
df["seniority"].value_counts()

na        519
senior    220
junior      3
Name: seniority, dtype: int64

In [ ]:
# Description length
df["desc_len"] = df["Job Description"].apply(lambda x: len(x.split()))
df["desc_len"]

0      354
1      671
2      490
3      516
4      378
      ... 
950    871
951    877
952    421
953    221
955    522
Name: desc_len, Length: 742, dtype: int64

In [ ]:
#Competitors number
df["num_comp"] = df["Competitors"].apply(lambda x: len(x.split(',')) if not pd.isna(x) else 0)
df["num_comp"].value_counts()

0    460
3    228
2     41
1     12
4      1
Name: num_comp, dtype: int64

In [ ]:
# employees number
def employees_num(size):
  """
  Size is taken from our data frame and is one of two formats:
  - '501 to 1000 employees'
  - 10000+ employees
  """
  if pd.isna(size) or "unknown" in size.lower():
    return pd.NA
  elif "to" in size:
    num1, num2 = size.split()[0], size.split()[2]
    return (int(num1) + int(num2))/2
  else:
    return int(size.split("+")[0])

#df["Size"].value_counts()
df["employees_num"] = df["Size"].apply(employees_num)
df["employees_num"].value_counts()

3000.5     150
750.5      134
10000.0    130
350.5      117
125.5       94
7500.5      76
25.5        31
Name: employees_num, dtype: int64

In [ ]:
#revenue amount

def get_revenue(revenue):
  """
  $10+ billion (USD) is taken from our data frame and is one of three formats:
  - $25 to $50 million (USD)
  - $10+ billion (USD)
  - Less than $1 million (USD)
  """
  if pd.isna(revenue) or "unknown" in revenue.lower():
    return pd.NA
  elif "to" in revenue:
    elements = revenue.split()
    if len(elements) == 5:
      num1 = int(elements[0][1:])
      num2 = int(elements[2][1:])
      if "million" in revenue:
        num1, num2 = 1e-3*num1, 1e-3*num2
    else:
      num1 = int(elements[0][1:])*1e-3
      num2 = int(elements[3][1:])

    return (num1+num2)/2
  elif revenue == "$10+ billion (USD)":
    return 10
  elif revenue == "Less than $1 million (USD)":
    return 0.001

#df["Revenue"].value_counts()
df["revenue_avr"] = df["Revenue"].apply(get_revenue)
df["revenue_avr"].value_counts()

10.0000    124
0.3000      91
1.5000      60
0.7500      57
0.0750      46
0.0375      40
3.5000      39
0.0175      32
7.5000      19
0.0075      18
0.0030       8
0.0010       4
Name: revenue_avr, dtype: int64

In [ ]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'min_salary', 'max_salary', 'avr_salary', 'company_txt',
       'age', 'job_state', 'same_state', 'python_yn', 'R_yn', 'sql_yn',
       'excel_yn', 'job_title', 'seniority', 'desc_len', 'num_comp',
       'employees_num', 'revenue_avr'],
      dtype='object')

In [ ]:
# upload cleaned data
columns = ['Location', 'Headquarters', 'Type of ownership', 'Industry', "Rating",
           'Sector', "Job Description", 'hourly', 'min_salary', 'max_salary', 'avr_salary', 'company_txt',
           'age', 'job_state', 'same_state', 'python_yn', 'R_yn', 'sql_yn',
           'excel_yn', 'job_title', 'seniority', 'desc_len', 'num_comp',
           'employees_num', 'revenue_avr']

df = df[columns]
df.to_csv("glassdoor_jobs_cleaned.csv", index=False)